## Main

In [1]:
#1
import pandas as pd  # Import pandas to handle structured data (CSV files)

# Load dataset containing news articles
df = pd.read_csv("dataset/data.csv")  

# Display basic information about the dataset (column names, data types, missing values, etc.)
print(df.info())  

# Display the first few rows of the dataset to check its structure
print(df.head())  

# We first load the data to see what we’re working with. 
# It includes headlines, news content, topics, and labels showing whether the news is real or fake.


In [2]:
#2
import re  # Regular expressions for text cleaning
import string  # String operations to remove punctuation
import nltk  # Natural Language Toolkit (NLTK) for text processing
from nltk.corpus import stopwords  # Import stop words to remove unnecessary words
from nltk.tokenize import word_tokenize  # Tokenization (splitting text into words)
from nltk.stem import WordNetLemmatizer  # Lemmatization (converting words to their root form)

# Download necessary NLTK data files (only needs to be run once)
nltk.download('punkt')  # Tokenizer data
nltk.download('stopwords')  # Stopwords data (common words like "the", "is", etc.)
nltk.download('wordnet')  # WordNet dictionary for lemmatization

# Initialize tools
stop_words = set(stopwords.words('english'))  # Set of stopwords in English
lemmatizer = WordNetLemmatizer()  # Initialize lemmatizer to get base form of words


In [3]:
#3
def clean_text(text):
    text = text.lower()  # Convert text to lowercase to maintain consistency
    text = re.sub(r'\d+', '', text)  # Remove numbers since they don't contribute to meaning
    text = re.sub(r'[^\w\s]', '', text)  # Remove all punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    tokens = word_tokenize(text)  # Tokenize the text (split into words)
    tokens = [word for word in tokens if word not in stop_words]  # Remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Convert words to their base form
    return " ".join(tokens)  # Join words back into a single string

# Apply text cleaning function to 'title' and 'text' columns, then combine them
df['cleaned_text'] = df['title'] + " " + df['text']
df['cleaned_text'] = df['cleaned_text'].apply(clean_text)

# Print the first few cleaned rows to verify text preprocessing
print(df['cleaned_text'].head())

# We clean the text by making everything lowercase, removing unnecessary words, 
# and simplifying words to their root form. This helps our model focus on important words.


In [4]:
## 4 Split Data for Training and Testing
from sklearn.model_selection import train_test_split  # Import function to split dataset

# Define features (text) and target variable (label: 0 = fake, 1 = real)
X = df['cleaned_text']
y = df['label']

# Split dataset into 80% training and 20% testing to evaluate model performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")  # Print number of training samples
print(f"Testing samples: {len(X_test)}")  # Print number of testing samples

# We split the data so the model can learn from one part and be tested on another.
# This helps us check if it works on new articles.

#5
from sklearn.feature_extraction.text import TfidfVectorizer  # Import TF-IDF vectorizer

# Convert text into numerical features using TF-IDF (Term Frequency - Inverse Document Frequency)
vectorizer = TfidfVectorizer(max_features=5000)  # Keep top 5000 most relevant words
X_train_tfidf = vectorizer.fit_transform(X_train)  # Transform training text into numerical data
X_test_tfidf = vectorizer.transform(X_test)  # Transform test text using the same vectorizer

print("TF-IDF matrix shape:", X_train_tfidf.shape)  # Print shape of TF-IDF matrix

# We turn words into numbers! TF-IDF gives higher importance to words that are rare and meaningful.

## 6 Train Random Forest Model
from sklearn.ensemble import RandomForestClassifier  # Import Random Forest classifier

# Train a Random Forest classifier to classify real vs. fake news
model = RandomForestClassifier(n_estimators=100, random_state=42)  # Use 100 decision trees
model.fit(X_train_tfidf, y_train)  # Train the model

# Test the model and print accuracy
accuracy = model.score(X_test_tfidf, y_test)
print(f"Model Accuracy: {accuracy:.2f}")

# We train a machine learning model using Random Forest to recognize patterns in the text 
# and predict if an article is fake or real. Then we test how well it performs.


In [5]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Get predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Display confusion matrix
plt.figure(figsize=(6,5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Fake News (0)", "Real News (1)"])
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix - TF-IDF + Random Forest")
plt.show()


In [6]:
#7
# Load new articles for validation
val_df = pd.read_csv("dataset/validation_data.csv")

# Clean the validation text
val_df['cleaned_text'] = (val_df['title'] + " " + val_df['text']).apply(clean_text)

# Convert validation text to TF-IDF format
X_val_tfidf = vectorizer.transform(val_df['cleaned_text'])

# Predict fake (0) or real (1) labels
val_df['label'] = model.predict(X_val_tfidf)

# Replace any remaining label 2 values with 0 (fake)
val_df['label'] = val_df['label'].apply(lambda x: 0 if x == 2 else x)

# Save predictions to a new CSV file
val_df.to_csv("dataset/validation_predictions.csv", index=False)

print("Predictions saved successfully!")

# We clean and process new articles, then use our trained model to predict if they are fake or real. 
# Finally, we save the results!


In [7]:
#8
from sklearn.metrics import classification_report  # Import function to check model performance

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Print classification report with precision, recall, and F1-score
print(classification_report(y_test, y_pred))

# We check how well the model performs using accuracy, precision, and recall. 
# These numbers tell us how reliable our fake news detector is!


## Extra to show more

In [8]:
#9
from collections import Counter  # Import Counter to count word occurrences
import matplotlib.pyplot as plt  # Import Matplotlib for plotting

# Get most common words in real and fake news articles
real_words = " ".join(df[df["label"] == 1]["cleaned_text"]).split()
fake_words = " ".join(df[df["label"] == 0]["cleaned_text"]).split()

real_counts = Counter(real_words).most_common(15)  # Top 15 words in real news
fake_counts = Counter(fake_words).most_common(15)  # Top 15 words in fake news

# Plot most common words in real vs. fake news
plt.figure(figsize=(12,5))
plt.bar(*zip(*real_counts), color='blue', label="Real News")
plt.bar(*zip(*fake_counts), color='red', alpha=0.7, label="Fake News")
plt.xticks(rotation=45)
plt.title("Most Common Words in Fake vs. Real News")
plt.legend()
plt.show()

# Fake news often uses dramatic words, while real news tends to have more neutral wording. 
# This chart shows the most frequent words in both categories.


In [9]:
from wordcloud import WordCloud  # Import WordCloud to generate visual representations of text data

# Generate Word Clouds for Real and Fake News
real_wc = WordCloud(width=800, height=400, background_color="white").generate(" ".join(real_words))  
fake_wc = WordCloud(width=800, height=400, background_color="black").generate(" ".join(fake_words))

# Create a figure for displaying both word clouds
plt.figure(figsize=(12,5))  

# Display the Real News word cloud
plt.subplot(1,2,1)  # Create a subplot (1 row, 2 columns, 1st plot)
plt.imshow(real_wc, interpolation="bilinear")  # Show the word cloud image
plt.axis("off")  # Remove axis labels for cleaner display
plt.title("Real News Word Cloud")  # Add title

# Display the Fake News word cloud
plt.subplot(1,2,2)  # Create a subplot (1 row, 2 columns, 2nd plot)
plt.imshow(fake_wc, interpolation="bilinear")  # Show the word cloud image
plt.axis("off")  # Remove axis labels for cleaner display
plt.title("Fake News Word Cloud")  # Add title

# Show the final plot with both word clouds
plt.show()

# Shows most used words in real vs. fake news
# Helps detect patterns in language use



In [10]:
from nltk.sentiment import SentimentIntensityAnalyzer  # Import sentiment analysis tool

# Download sentiment analysis tool
nltk.download("vader_lexicon")  
sia = SentimentIntensityAnalyzer()  

# Compute sentiment scores for each article
df["sentiment"] = df["cleaned_text"].apply(lambda text: sia.polarity_scores(text)["compound"])  

# Plot sentiment distribution
plt.figure(figsize=(10,5))
plt.hist(df[df["label"] == 1]["sentiment"], bins=20, alpha=0.7, label="Real News", color="blue")  
plt.hist(df[df["label"] == 0]["sentiment"], bins=20, alpha=0.7, label="Fake News", color="red")  
plt.title("Sentiment Distribution in Fake vs. Real News")  
plt.xlabel("Sentiment Score")  
plt.ylabel("Count")  
plt.legend()  
plt.show()

# Measures the emotional tone of articles
# Shows if fake news is more negative or emotional




In [11]:
from sklearn.ensemble import RandomForestClassifier  # Import Random Forest model
from sklearn.linear_model import LogisticRegression  # Import Logistic Regression model
from sklearn.naive_bayes import MultinomialNB  # Import Naive Bayes model
from sklearn.metrics import accuracy_score  # Import accuracy evaluation

# Define models to test
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=500),  # Increased iterations for better convergence
    "Naive Bayes": MultinomialNB()
}

# Train and evaluate each model
results = {}
for name, model in models.items():
    model.fit(X_train_tfidf, y_train)  # Train model
    y_pred = model.predict(X_test_tfidf)  # Make predictions
    accuracy = accuracy_score(y_test, y_pred)  # Calculate accuracy
    results[name] = accuracy  
    print(f"{name} Accuracy: {accuracy:.2f}")  

# Plot model comparison
plt.figure(figsize=(8,4))
plt.bar(results.keys(), results.values(), color=["blue", "green", "orange"])
plt.title("Model Comparison")  
plt.ylabel("Accuracy")  
plt.ylim(0.9, 1.01)  # Adjust y-axis to highlight high accuracy
plt.show()

# Compares Random Forest, Logistic Regression, and Naive Bayes
# Removes SVM to speed up training while keeping high accuracy


In [12]:
import joblib

# Save trained model
joblib.dump(model, "random_forest_model.pkl")

# Save TF-IDF vectorizer
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


['tfidf_vectorizer.pkl']